In [1]:
import struct
from math import sqrt, modf

from bitstring import BitArray

In [2]:
def prime(a):
    return not (a < 2 or any(a % x == 0 for x in range(2, int(a**0.5) + 1)))

def primes_below(n):
    return [i for i in range(n) if prime(i)]

In [3]:
def first_n_primes(n):
    P = []
    i = 2
    while len(P) < n:
        if prime(i):
            P.append(i)
        i = i + 1
    return P

In [4]:
def fractional_float_to_int(n):
    f, _ = modf(n)  # Fractional part of n.
    return int(f * (1 << 32))  # Multiply by 2^32 then convert to int.

def fractional_float_to_hex(n):
    f, _ = modf(n)  # Fractional part of n.
    f = int(f * (1 << 32))  # Multiply by 2^32 then convert to int.
    return hex(f)

In [5]:
ror32 = lambda x, n: (x >> n) | (x << (32 - n))
Ch = lambda E, F, G: (E & F) ^ ((~E) & G)
Ma = lambda A, B, C: (A & B) ^ (A & C) ^ (B & C)
Σ0 = lambda A: ror32(A, 2) ^ ror32(A, 13) ^ ror32(A, 22)
Σ1 = lambda E: ror32(E, 6) ^ ror32(E, 11) ^ ror32(E, 25)

In [47]:
def SHA256(message):
    # Mask that allows us to perform (mod 2^32) by &'ing with an int.
    MASK32 = 0xffffffff
    
    # Initialize hash values:
    # (first 32 bits of the fractional parts of the square roots of the first 8 primes 2..19):
    _h = 8 * [0]
    for i, p in enumerate(first_n_primes(8)):
        _h[i] = fractional_float_to_int(sqrt(p))
        
    # Initialize array of round constants:
    # (first 32 bits of the fractional parts of the cube roots of the first 64 primes 2..311):
    _k = 64 * [0]
    for i, p in enumerate(first_n_primes(64)):
        _k[i] = fractional_float_to_int(p**(1/3))
        
    # Message to be hashed.
    M = BitArray(bytes(message, "utf-8"))

    # Pre-processing (Padding):
    # begin with the original message of length L bits
    L = len(M)

    # append a single '1' bit
    M.append(BitArray(bin="1"))

    # append K '0' bits, where K is the minimum number >= 0 such that L + 1 + K + 64 is a multiple of 512
    def pad_size(L):
        return (512 - ((L % 512) + 1 + 64)) % 512

    K = pad_size(L)
    M = M + BitArray(K)

    # append L as a 64-bit big-endian integer, making the total post-processed length a multiple of 512 bits
    M = M + BitArray(int=L, length=64)
    
    # Process the message in successive 512-bit chunks:
    # break message into 512-bit chunks    
    for chunk in M.cut(512):
        # create a 64-entry message schedule array w[0..63] of 32-bit words
        # (The initial values in w[0..63] don't matter, so many implementations zero them here)
        # copy chunk into first 16 words w[0..15] of the message schedule array
        w = 64 * [0]
        for i in range(16):
            w[i] = chunk[32*i:32*i+32].int & MASK32

        # Extend the first 16 words into the remaining 48 words w[16..63] of the message schedule array:
        for i in range(16, 64):
            s0 = ror32(w[i-15], 7) ^ ror32(w[i-15], 18) ^ (w[i-15] >> 3)
            s1 = ror32(w[i-2], 17) ^ ror32(w[i-2], 19) ^ (w[i-2] >> 10)
            w[i] = (w[i-16] + s0 + w[i-7] + s1) & MASK32
        
        # Initialize working variables to current hash value:
        a, b, c, d, e, f, g, h = _h

        # Compression function main loop:
        for i in range(64):
            t1 = h + Σ1(e) + Ch(e, f, g) + _k[i] + w[i]
            t2 = Σ0(a) + Ma(a, b, c)

            h = g
            g = f
            f = e
            e = (d + t1) & MASK32
            d = c
            c = b
            b = a
            a = (t1 + t2) & MASK32
        
        # Add the compressed chunk to the current hash value:
        _h[0] = (_h[0] + a) & MASK32
        _h[1] = (_h[1] + b) & MASK32
        _h[2] = (_h[2] + c) & MASK32
        _h[3] = (_h[3] + d) & MASK32
        _h[4] = (_h[4] + e) & MASK32
        _h[5] = (_h[5] + f) & MASK32
        _h[6] = (_h[6] + g) & MASK32
        _h[7] = (_h[7] + h) & MASK32
        
    digest = sum([BitArray(uint=n, length=32) for n in _h])
    return digest.hex

In [48]:
import hashlib

SHA256("")

'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855'

In [49]:
SHA256("") == hashlib.sha256(b"").hexdigest()

True

In [50]:
SHA256("The quick brown fox jumps over the lazy dog")

'd7a8fbb307d7809469ca9abcb0082e4f8d5651e46d3cdb762d02d0bf37c9e592'

In [51]:
SHA256("The quick brown fox jumps over the lazy dog") == hashlib.sha256(b"The quick brown fox jumps over the lazy dog").hexdigest()

True

In [55]:
SHA256(12345 * "LOL")

'c9727681e0864afe7168d2b2a674ab0bd5209dc42f75182bf1a5b8639efa7a49'

In [60]:
hashlib.sha256(bytes(12345 * "LOL", "utf-8")).hexdigest()

'c9727681e0864afe7168d2b2a674ab0bd5209dc42f75182bf1a5b8639efa7a49'